# Figure 2: Do similar OAX and MHW appear in obs and FOSI?

In [ ]:
# general use packages
%matplotlib inline
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import numpy as np

import regionmask
import geopandas as gp
import cartopy.crs as ccrs
import cartopy.feature as cfeature

## not-rolling (calculate here)

In [ ]:
obs = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/OceanSODA-ETHZ_GRaCER_v2021a_1982-2020.nc')

In [ ]:
fosi_omega = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/FOSI/omega_arag.monthly.surface.regrid.nc')['omega_arag']
fosi_temp = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/FOSI/TEMP.monthly.surface.regrid.nc')['TEMP']
fosi_h = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/FOSI/pH_3D.monthly.surface.regrid.nc')['pH_3D']; fosi_h = 10**(-fosi_h)

In [ ]:
fosi_omega['time'] = pd.date_range("1958-01", "2020-12", freq="MS")
fosi_temp['time'] = pd.date_range("1958-01", "2020-12", freq="MS")
fosi_h['time'] = pd.date_range("1958-01", "2020-12", freq="MS")

In [ ]:
obs_omega = obs.omega_ar
obs_temp = obs.temperature
obs_h = 10**(-obs.ph_total)

In [ ]:
obs_omega, fosi_omega = xr.align(obs_omega, fosi_omega)
obs_temp, fosi_temp = xr.align(obs_temp, fosi_temp)
obs_h, fosi_h = xr.align(obs_h, fosi_h)

In [ ]:
mask = np.isnan(obs_omega.isel(time=0))#obs_omega.isel(time=0).where(obs_omega.isel(time=0)==np.NaN,obs_omega.isel(time=0))
mask = mask.drop('time')

In [ ]:
fosi_omega = fosi_omega.where(mask == 0, np.NaN)
fosi_temp = fosi_temp.where(mask == 0, np.NaN)
fosi_h = fosi_h.where(mask == 0, np.NaN)

### remove climatology and trend

In [ ]:
def detrend_linear(dat, dim):
    """ linear detrend dat along the axis dim """
    params = dat.polyfit(dim=dim, deg=1)
    fit = xr.polyval(dat[dim], params.polyfit_coefficients)
    dat = dat-fit
    return dat

In [ ]:
# climatology
fosi_omega = fosi_omega.groupby('time.month') - fosi_omega.groupby('time.month').mean()
fosi_temp = fosi_temp.groupby('time.month') - fosi_temp.groupby('time.month').mean()
fosi_h = fosi_h.groupby('time.month') - fosi_h.groupby('time.month').mean()

obs_omega = obs_omega.groupby('time.month') - obs_omega.groupby('time.month').mean()
obs_temp = obs_temp.groupby('time.month') - obs_temp.groupby('time.month').mean()
obs_h = obs_h.groupby('time.month') - obs_h.groupby('time.month').mean()

In [ ]:
# trend
fosi_omega = detrend_linear(fosi_omega,'time')
fosi_temp = detrend_linear(fosi_temp,'time')
fosi_h = detrend_linear(fosi_h,'time')

obs_omega = detrend_linear(obs_omega,'time')
obs_temp = detrend_linear(obs_temp,'time')
obs_h = detrend_linear(obs_h,'time')

## Load pre-Defined Extremes

In [ ]:
obs_temp_extreme = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/TEMP.obs.rolling.thold.Rolling.nc')['threshold']
obs_omega_extreme = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/omega_arag.obs.rolling.thold.Rolling.nc')['threshold']
obs_H_extreme = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/H+.obs.rolling.thold.Rolling.nc')['threshold']


In [ ]:
fosi_temp_extreme = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/TEMP.fosi.rolling.thold.Rolling.nc')['threshold']
fosi_omega_extreme = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/omega_arag.fosi.rolling.thold.Rolling.nc')['threshold']
fosi_H_extreme = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/H+.fosi.rolling.thold.Rolling.nc')['threshold']


In [ ]:
ds = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/OceanSODA-ETHZ_GRaCER_v2021a_1982-2020.nc')['temperature'].isel(time=0).drop('time')
mask = ds.where(ds.lat < 65)

In [ ]:
fosi_temp_extreme = fosi_temp_extreme.where(np.isnan(mask) == 0, np.NaN)
fosi_omega_extreme = fosi_omega_extreme.where(np.isnan(mask) == 0, np.NaN)
fosi_H_extreme = fosi_H_extreme.where(np.isnan(mask) == 0, np.NaN)

## Plot

In [ ]:
# plot them together!
f, ax = plt.subplots(3,1,figsize=(6,8),subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

xr.corr(obs_temp,fosi_temp,dim='time').plot(ax=ax[0],levels=np.arange(-1,1.1,0.1),extend='neither',transform = ccrs.PlateCarree(),cmap='coolwarm',add_colorbar=False)
ax[0].add_feature(cfeature.LAND, color='k', zorder=3)
ax[0].set_title('SST',fontsize=15)

xr.corr(obs_omega,fosi_omega,dim='time').plot(ax=ax[1],levels=np.arange(-1,1.1,0.1),extend='neither',transform = ccrs.PlateCarree(),cmap='coolwarm',add_colorbar=False)
ax[1].add_feature(cfeature.LAND, color='k', zorder=3)
ax[1].set_title('$\Omega_{arag}$',fontsize=15)

im = xr.corr(obs_h,fosi_h,dim='time').plot(ax=ax[2],levels=np.arange(-1,1.1,0.1),extend='neither',transform = ccrs.PlateCarree(),cmap='coolwarm',add_colorbar=False)
ax[2].add_feature(cfeature.LAND, color='k', zorder=3)
ax[2].set_title('H$^+$',fontsize=15)

# plt.tight_layout()

f.subplots_adjust(right=0.8)
cbar_ax = f.add_axes([0.83, 0.2, 0.03, 0.60])
cbar = f.colorbar(im, cax=cbar_ax, ticks=[-1,-0.5,0,0.5,1])
cbar.ax.tick_params(labelsize=8)
cbar.set_label('Correlation Coefficient', rotation=270,fontsize=14)

f.savefig('./figures/Review.Obs.FOSI.Correlation.Variability.pdf')

In [ ]:
# plot them together!
f, ax = plt.subplots(3,1,figsize=(6,8),subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

xr.corr(obs_temp_extreme,fosi_temp_extreme,dim='time').plot(ax=ax[0],levels=np.arange(-1,1.1,0.1),extend='neither',transform = ccrs.PlateCarree(),cmap='coolwarm',add_colorbar=False)
ax[0].add_feature(cfeature.LAND, color='k', zorder=3)
ax[0].set_title('MHW',fontsize=15)

xr.corr(obs_omega_extreme,fosi_omega_extreme,dim='time').plot(ax=ax[1],levels=np.arange(-1,1.1,0.1),extend='neither',transform = ccrs.PlateCarree(),cmap='coolwarm',add_colorbar=False)
ax[1].add_feature(cfeature.LAND, color='k', zorder=3)
ax[1].set_title('OAX ($\Omega_{arag}$)',fontsize=15)

im = xr.corr(obs_H_extreme,fosi_H_extreme,dim='time').plot(ax=ax[2],levels=np.arange(-1,1.1,0.1),extend='neither',transform = ccrs.PlateCarree(),cmap='coolwarm',add_colorbar=False)
ax[2].add_feature(cfeature.LAND, color='k', zorder=3)
ax[2].set_title('OAX (H$^+$)',fontsize=15)

# plt.tight_layout()

f.subplots_adjust(right=0.8)
cbar_ax = f.add_axes([0.83, 0.2, 0.03, 0.60])
cbar = f.colorbar(im, cax=cbar_ax, ticks=[-1,-0.5,0,0.5,1])
cbar.ax.tick_params(labelsize=8)
cbar.set_label('Correlation Coefficient', rotation=270,fontsize=14)

f.savefig('./figures/Figure2.Obs.FOSI.Correlation.pdf')

In [ ]:
# MHW and OA (omega)

obs_mhw_omega = (obs_temp_extreme.astype('int') + obs_omega_extreme.astype('int'))
obs_mhw_omega = ~np.isnan(obs_mhw_omega.where(obs_mhw_omega == 2))

fosi_mhw_omega = (fosi_temp_extreme.astype('int') + fosi_omega_extreme.astype('int'))
fosi_mhw_omega = ~np.isnan(fosi_mhw_omega.where(fosi_mhw_omega == 2))

In [ ]:
# MHW and OA (H+)

obs_mhw_h = (obs_temp_extreme.astype('int') + obs_h_extreme.astype('int'))
obs_mhw_h = ~np.isnan(obs_mhw_h.where(obs_mhw_h == 2))

fosi_mhw_h = (fosi_temp_extreme.astype('int') + fosi_h_extreme.astype('int'))
fosi_mhw_h = ~np.isnan(fosi_mhw_h.where(fosi_mhw_h == 2))

In [ ]:
# OA compound

obs_h_omega = (obs_omega_extreme.astype('int') + obs_h_extreme.astype('int'))
obs_h_omega = ~np.isnan(obs_h_omega.where(obs_h_omega == 2))

fosi_h_omega = (fosi_omega_extreme.astype('int') + fosi_h_extreme.astype('int'))
fosi_h_omega = ~np.isnan(fosi_h_omega.where(fosi_h_omega == 2))

In [ ]:
# plot them together!
f, ax = plt.subplots(3,1,figsize=(6,8),subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

xr.corr(obs_mhw_omega,fosi_mhw_omega,dim='time').plot(ax=ax[0],levels=np.arange(-1,1.1,0.1),extend='neither',transform = ccrs.PlateCarree(),cmap='coolwarm')
ax[0].add_feature(cfeature.LAND, color='k', zorder=3)
ax[0].set_title('Correlation of MHW + OAX ($\Omega_{arag}$): FOSI and Obs.',fontsize=15)

xr.corr(obs_mhw_h,fosi_mhw_h,dim='time').plot(ax=ax[1],levels=np.arange(-1,1.1,0.1),extend='neither',transform = ccrs.PlateCarree(),cmap='coolwarm')
ax[1].add_feature(cfeature.LAND, color='k', zorder=3)
ax[1].set_title('Correlation of MHW and OAX (H$^+$): FOSI and Obs.',fontsize=15)

xr.corr(obs_h_omega,fosi_h_omega,dim='time').plot(ax=ax[2],levels=np.arange(-1,1.1,0.1),extend='neither',transform = ccrs.PlateCarree(),cmap='coolwarm')
ax[2].add_feature(cfeature.LAND, color='k', zorder=3)
ax[2].set_title('Correlation of OAX (H$^+$) and OAX ($\Omega_{arag}$): FOSI and Obs.',fontsize=15)

plt.tight_layout()

In [ ]:
obs_mhw_h.isel(time=100).plot()

In [ ]:
fosi_mhw_h.sel(lat=-0.5,lon=-140.5).plot()

In [ ]:
var = 'TEMP'
var2 = 'TEMP'
depth = 'surface'
# init = '05'

smyle02 = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/thresholds/'+var +  '.monthly.' + depth + '.02.binary.Rolling.full.2.nc')['binary']
smyle02_time = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var2+'.monthly.02.time.nc')#.sel(L=slice(1,12))

smyle05 = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/thresholds/'+var +  '.monthly.' + depth + '.05.binary.Rolling.full.2.nc')['binary']
smyle05_time = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var2+'.monthly.05.time.nc')#.sel(L=slice(1,12))

smyle08 = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/thresholds/'+var +  '.monthly.' + depth + '.08.binary.Rolling.full.2.nc')['binary']
smyle08_time = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var2+'.monthly.08.time.nc')#.sel(L=slice(1,12))

smyle11 = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/thresholds/'+var +  '.monthly.' + depth + '.11.binary.Rolling.full.2.nc')['binary']
smyle11_time = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var2+'.monthly.11.time.nc')#.sel(L=slice(1,12))

smyle02 = smyle02.expand_dims("init"); smyle02_time  = smyle02_time.expand_dims("init")
smyle05 = smyle05.expand_dims("init"); smyle05_time  = smyle05_time.expand_dims("init")
smyle08 = smyle08.expand_dims("init"); smyle08_time  = smyle08_time.expand_dims("init")
smyle11 = smyle11.expand_dims("init"); smyle11_time  = smyle11_time.expand_dims("init")

smyle02['init'] = ['02']; smyle02_time['init'] = ['02']
smyle05['init'] = ['05']; smyle05_time['init'] = ['05']
smyle08['init'] = ['08']; smyle08_time['init'] = ['08']
smyle11['init'] = ['11']; smyle11_time['init'] = ['11']

smyle = xr.concat([smyle02,smyle05,smyle08,smyle11],'init')
smyle_time = xr.concat([smyle02_time,smyle05_time,smyle08_time,smyle11_time],'init')

In [ ]:
smyle